In [32]:
!pip3 install awswrangler

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [1]:
import logging
import sys

import boto3
from botocore.exceptions import ClientError

In [2]:
logger = logging.getLogger(__name__)
client = boto3.client('sqs')

In [26]:
# Retrieve Data from SQS
def receive_messages(queueUrl, max_num, visibilityTimeout, waitTimeSeconds):
    try:
        messages = client.receive_message(
            QueueUrl = queueUrl,
            MessageAttributeNames = ['All'],
            MaxNumberOfMessages = max_num,
            VisibilityTimeout = visibilityTimeout,
            WaitTimeSeconds = waitTimeSeconds
        )
    except ClientError as error:
        logger.exception("Couldn't receive messages from queue: %s", queueUrl)
        raise error
    else:
        return messages

# Common logic - Unpack Data 
def unpack_messages(messages):
    message_bodys = []
    try: 
        for msg in messages['Messages']:
            message_bodys.append(msg['Body'])
    except:
        pass
    return message_bodys

def check_queue_empty(queueUrl):
    response = client.get_queue_attributes(
    QueueUrl= queueUrl,
    AttributeNames=[
        'ApproximateNumberOfMessages'
        ]
    )
    return int(response['Attributes']['ApproximateNumberOfMessages'])

def delete_messages(queueUrl, messages, flag):
    try:
        entries = [{
            'Id': str(ind),
            'ReceiptHandle': msg['ReceiptHandle']
        } for ind, msg in enumerate(messages['Messages'])]
        response = client.delete_message_batch(QueueUrl=queueUrl,Entries=entries)
        if 'Successful' in response:
            for msg_meta in response['Successful']:
                logger.info("Deleted %s", messages['Messages'][int(msg_meta['Id'])]['ReceiptHandle'])
        if 'Failed' in response:
            for msg_meta in response['Failed']:
                logger.warning(
                    "Could not delete %s",
                    messages['Messages'][int(msg_meta['Id'])]['ReceiptHandle']
                )
        return True
    except:
        if check_queue_empty(queueUrl) == 0:
            logger.info("Queue is empty now.")
            return False
        else:
            delete_messages1(queueUrl, messages, flag)

In [27]:
queueUrl = 'https://sqs.us-east-1.amazonaws.com/521981389199/sp-traffic-mock'

#Max batch size for a sqs call
max_num = 10

#A period of time during which Amazon SQS prevents other consumers from receiving and processing the message
visibilityTimeout = 90

#A period of time call waits for a message to arrive in the queue before returning
waitTimeSeconds = 0

#S3 bucket location for cleaned data
s3_location = 'S3://dth-bucket/'

In [28]:
import json

more_messages = True
messages = []
while more_messages:
    sqs_receive_messages = receive_messages(queueUrl, max_num, visibilityTimeout, waitTimeSeconds)
    messages.append(unpack_messages(sqs_receive_messages))
    
    if sqs_receive_messages:
        print (more_messages)
        more_messages = delete_messages(queueUrl, sqs_receive_messages, more_messages)  
        
    else:
        more_messages = False


<class 'dict'>
True
<class 'dict'>
True
<class 'dict'>
True
<class 'dict'>
True
<class 'dict'>
True
<class 'dict'>
True
<class 'dict'>
True


In [29]:
print(type(messages))
print(len(messages))

i = 0
j = 0 
for msg in messages:
    for x_msg in msg:
        j = j + 1
    i = i + j
    
print (i)

<class 'list'>
7
66


In [30]:
import pandas as pd 
import json

def create_df():
    if json.loads(messages[0][0])['dataset_id'] == 'sp-traffic':
        df = pd.DataFrame({c: pd.Series(dtype=t) for c, t in {'idempotency_id': 'str', 
                                                              'dataset_id': 'str', 
                                                              'marketplace_id': 'str', 
                                                              'currency': 'str', 
                                                              'advertiser_id': 'str', 
                                                              'campaign_id': 'str', 
                                                              'ad_group_id': 'str', 
                                                              'ad_id': 'str', 
                                                              'keyword_id': 'str', 
                                                              'keyword_text': 'str', 
                                                              'match_type': 'str', 
                                                              'placement': 'str', 
                                                              'time_window_start': 'str', 
                                                              'clicks': 'int', 
                                                              'impressions': 'int', 
                                                              'cost': 'float'}.items()})
    # TODO other stream type
    #if json.loads(messages[0])['dataset_id'] == 'sp-conversion':
    #if json.loads(messages[0])['dataset_id'] == 'budget-usage':
    return df

df = create_df()

for msg in messages:
    for x_msg in msg:
        to_json = json.loads(x_msg)
        try : 
            if to_json['time_window_start']:
                df_msg = pd.json_normalize(json.loads(x_msg))
                df = pd.concat([df, df_msg], ignore_index=True)
        except:
            pass
df

,idempotency_id,dataset_id,marketplace_id,currency,advertiser_id,campaign_id,ad_group_id,ad_id,keyword_id,keyword_text,match_type,placement,time_window_start,clicks,impressions,cost
0,b93d056d-df33-4778-88b1-52986cbd1213,sp-traffic,ATVPDKIKX0DER,USD,A2UQYYL1FV7LGU,185402650650808,230449422777252,167430181164441,25420145967171,substitutes,TARGETING_EXPRESSION_PREDEFINED,Detail Page on-Amazon,2022-09-10T06:00:00-07:00,0,-11,0.00
1,53ec9fb4-00b1-4b0a-8ecc-63e1250fdc08,sp-traffic,ATVPDKIKX0DER,USD,A2UQYYL1FV7LGU,185402650650808,230449422777252,167430181164441,25420145967171,substitutes,TARGETING_EXPRESSION_PREDEFINED,Detail Page on-Amazon,2022-09-10T07:00:00-07:00,0,-2,0.00
2,e96bfd9b-626b-4f96-9d98-04561b68f233,sp-traffic,ATVPDKIKX0DER,USD,A2UQYYL1FV7LGU,185402650650808,230449422777252,167430181164441,236652758809586,close-match,TARGETING_EXPRESSION_PREDEFINED,Detail Page on-Amazon,2022-09-10T07:00:00-07:00,0,-11,0.00
3,99c1fb4f-427a-48fc-aaa7-1c7effe75a51,sp-traffic,ATVPDKIKX0DER,USD,A2UQYYL1FV7LGU,185402650650808,230449422777252,167430181164441,236652758809586,close-match,TARGETING_EXPRESSION_PREDEFINED,Detail Page on-Amazon,2022-09-10T02:00:00-07:00,0,-13,0.00
4,b6756e3a-8c25-49f1-8c8d-639e49678679,sp-traffic,ATVPDKIKX0DER,USD,A2UQYYL1FV7LGU,185402650650808,230449422777252,167430181164441,25420145967171,substitutes,TARGETING_EXPRESSION_PREDEFINED,Detail Page on-Amazon,2022-09-10T18:00:00-07:00,0,4,0.00
5,aaf7c14c-5c99-43a3-84c9-19794c74dc24,sp-traffic,ATVPDKIKX0DER,USD,A2UQYYL1FV7LGU,185402650650808,230449422777252,167430181164441,236652758809586,close-match,TARGETING_EXPRESSION_PREDEFINED,Detail Page on-Amazon,2022-09-10T03:00:00-07:00,0,-4,0.00
6,2d0ff425-6d4d-4c24-9559-64de7eb59d29,sp-traffic,ATVPDKIKX0DER,USD,A2UQYYL1FV7LGU,185402650650808,230449422777252,167430181164441,236652758809586,close-match,TARGETING_EXPRESSION_PREDEFINED,Detail Page on-Amazon,2022-09-10T05:00:00-07:00,0,-18,0.00
7,34ee598f-a298-4acc-a557-190e74aef3bf,sp-traffic,ATVPDKIKX0DER,USD,A2UQYYL1FV7LGU,185402650650808,230449422777252,167430181164441,25420145967171,substitutes,TARGETING_EXPRESSION_PREDEFINED,Detail Page on-Amazon,2022-09-10T03:00:00-07:00,0,-3,0.00
8,9e6b4357-6da3-45f3-a6ac-fd4a9bbd49b1,sp-traffic,ATVPDKIKX0DER,USD,A2UQYYL1FV7LGU,185402650650808,230449422777252,167430181164441,236652758809586,close-match,TARGETING_EXPRESSION_PREDEFINED,Detail Page on-Amazon,2022-09-10T06:00:00-07:00,0,-29,0.00
9,b2347ae8-e52f-4515-b5cc-c055fca6da73,sp-traffic,ATVPDKIKX0DER,USD,A2UQYYL1FV7LGU,185402650650808,230449422777252,167430181164441,25420145967171,substitutes,TARGETING_EXPRESSION_PREDEFINED,Detail Page on-Amazon,2022-09-10T04:00:00-07:00,0,-7,0.00


In [31]:
df = df.drop_duplicates()
df

,idempotency_id,dataset_id,marketplace_id,currency,advertiser_id,campaign_id,ad_group_id,ad_id,keyword_id,keyword_text,match_type,placement,time_window_start,clicks,impressions,cost
0,b93d056d-df33-4778-88b1-52986cbd1213,sp-traffic,ATVPDKIKX0DER,USD,A2UQYYL1FV7LGU,185402650650808,230449422777252,167430181164441,25420145967171,substitutes,TARGETING_EXPRESSION_PREDEFINED,Detail Page on-Amazon,2022-09-10T06:00:00-07:00,0,-11,0.00
1,53ec9fb4-00b1-4b0a-8ecc-63e1250fdc08,sp-traffic,ATVPDKIKX0DER,USD,A2UQYYL1FV7LGU,185402650650808,230449422777252,167430181164441,25420145967171,substitutes,TARGETING_EXPRESSION_PREDEFINED,Detail Page on-Amazon,2022-09-10T07:00:00-07:00,0,-2,0.00
2,e96bfd9b-626b-4f96-9d98-04561b68f233,sp-traffic,ATVPDKIKX0DER,USD,A2UQYYL1FV7LGU,185402650650808,230449422777252,167430181164441,236652758809586,close-match,TARGETING_EXPRESSION_PREDEFINED,Detail Page on-Amazon,2022-09-10T07:00:00-07:00,0,-11,0.00
3,99c1fb4f-427a-48fc-aaa7-1c7effe75a51,sp-traffic,ATVPDKIKX0DER,USD,A2UQYYL1FV7LGU,185402650650808,230449422777252,167430181164441,236652758809586,close-match,TARGETING_EXPRESSION_PREDEFINED,Detail Page on-Amazon,2022-09-10T02:00:00-07:00,0,-13,0.00
4,b6756e3a-8c25-49f1-8c8d-639e49678679,sp-traffic,ATVPDKIKX0DER,USD,A2UQYYL1FV7LGU,185402650650808,230449422777252,167430181164441,25420145967171,substitutes,TARGETING_EXPRESSION_PREDEFINED,Detail Page on-Amazon,2022-09-10T18:00:00-07:00,0,4,0.00
5,aaf7c14c-5c99-43a3-84c9-19794c74dc24,sp-traffic,ATVPDKIKX0DER,USD,A2UQYYL1FV7LGU,185402650650808,230449422777252,167430181164441,236652758809586,close-match,TARGETING_EXPRESSION_PREDEFINED,Detail Page on-Amazon,2022-09-10T03:00:00-07:00,0,-4,0.00
6,2d0ff425-6d4d-4c24-9559-64de7eb59d29,sp-traffic,ATVPDKIKX0DER,USD,A2UQYYL1FV7LGU,185402650650808,230449422777252,167430181164441,236652758809586,close-match,TARGETING_EXPRESSION_PREDEFINED,Detail Page on-Amazon,2022-09-10T05:00:00-07:00,0,-18,0.00
7,34ee598f-a298-4acc-a557-190e74aef3bf,sp-traffic,ATVPDKIKX0DER,USD,A2UQYYL1FV7LGU,185402650650808,230449422777252,167430181164441,25420145967171,substitutes,TARGETING_EXPRESSION_PREDEFINED,Detail Page on-Amazon,2022-09-10T03:00:00-07:00,0,-3,0.00
8,9e6b4357-6da3-45f3-a6ac-fd4a9bbd49b1,sp-traffic,ATVPDKIKX0DER,USD,A2UQYYL1FV7LGU,185402650650808,230449422777252,167430181164441,236652758809586,close-match,TARGETING_EXPRESSION_PREDEFINED,Detail Page on-Amazon,2022-09-10T06:00:00-07:00,0,-29,0.00
9,b2347ae8-e52f-4515-b5cc-c055fca6da73,sp-traffic,ATVPDKIKX0DER,USD,A2UQYYL1FV7LGU,185402650650808,230449422777252,167430181164441,25420145967171,substitutes,TARGETING_EXPRESSION_PREDEFINED,Detail Page on-Amazon,2022-09-10T04:00:00-07:00,0,-7,0.00


In [33]:
# Add partition columns
import datetime
year = datetime.datetime.now().year
month = datetime.datetime.now().month
day = datetime.datetime.now().day

print (year, month, day)

df.insert(len(df.columns) ,'year', year)
df.insert(len(df.columns) ,'month', month)
df.insert(len(df.columns) ,'day', day)

df

2022 9 12


,idempotency_id,dataset_id,marketplace_id,currency,advertiser_id,campaign_id,ad_group_id,ad_id,keyword_id,keyword_text,match_type,placement,time_window_start,clicks,impressions,cost,year,month,day
0,b93d056d-df33-4778-88b1-52986cbd1213,sp-traffic,ATVPDKIKX0DER,USD,A2UQYYL1FV7LGU,185402650650808,230449422777252,167430181164441,25420145967171,substitutes,TARGETING_EXPRESSION_PREDEFINED,Detail Page on-Amazon,2022-09-10T06:00:00-07:00,0,-11,0.00,2022,9,12
1,53ec9fb4-00b1-4b0a-8ecc-63e1250fdc08,sp-traffic,ATVPDKIKX0DER,USD,A2UQYYL1FV7LGU,185402650650808,230449422777252,167430181164441,25420145967171,substitutes,TARGETING_EXPRESSION_PREDEFINED,Detail Page on-Amazon,2022-09-10T07:00:00-07:00,0,-2,0.00,2022,9,12
2,e96bfd9b-626b-4f96-9d98-04561b68f233,sp-traffic,ATVPDKIKX0DER,USD,A2UQYYL1FV7LGU,185402650650808,230449422777252,167430181164441,236652758809586,close-match,TARGETING_EXPRESSION_PREDEFINED,Detail Page on-Amazon,2022-09-10T07:00:00-07:00,0,-11,0.00,2022,9,12
3,99c1fb4f-427a-48fc-aaa7-1c7effe75a51,sp-traffic,ATVPDKIKX0DER,USD,A2UQYYL1FV7LGU,185402650650808,230449422777252,167430181164441,236652758809586,close-match,TARGETING_EXPRESSION_PREDEFINED,Detail Page on-Amazon,2022-09-10T02:00:00-07:00,0,-13,0.00,2022,9,12
4,b6756e3a-8c25-49f1-8c8d-639e49678679,sp-traffic,ATVPDKIKX0DER,USD,A2UQYYL1FV7LGU,185402650650808,230449422777252,167430181164441,25420145967171,substitutes,TARGETING_EXPRESSION_PREDEFINED,Detail Page on-Amazon,2022-09-10T18:00:00-07:00,0,4,0.00,2022,9,12
5,aaf7c14c-5c99-43a3-84c9-19794c74dc24,sp-traffic,ATVPDKIKX0DER,USD,A2UQYYL1FV7LGU,185402650650808,230449422777252,167430181164441,236652758809586,close-match,TARGETING_EXPRESSION_PREDEFINED,Detail Page on-Amazon,2022-09-10T03:00:00-07:00,0,-4,0.00,2022,9,12
6,2d0ff425-6d4d-4c24-9559-64de7eb59d29,sp-traffic,ATVPDKIKX0DER,USD,A2UQYYL1FV7LGU,185402650650808,230449422777252,167430181164441,236652758809586,close-match,TARGETING_EXPRESSION_PREDEFINED,Detail Page on-Amazon,2022-09-10T05:00:00-07:00,0,-18,0.00,2022,9,12
7,34ee598f-a298-4acc-a557-190e74aef3bf,sp-traffic,ATVPDKIKX0DER,USD,A2UQYYL1FV7LGU,185402650650808,230449422777252,167430181164441,25420145967171,substitutes,TARGETING_EXPRESSION_PREDEFINED,Detail Page on-Amazon,2022-09-10T03:00:00-07:00,0,-3,0.00,2022,9,12
8,9e6b4357-6da3-45f3-a6ac-fd4a9bbd49b1,sp-traffic,ATVPDKIKX0DER,USD,A2UQYYL1FV7LGU,185402650650808,230449422777252,167430181164441,236652758809586,close-match,TARGETING_EXPRESSION_PREDEFINED,Detail Page on-Amazon,2022-09-10T06:00:00-07:00,0,-29,0.00,2022,9,12
9,b2347ae8-e52f-4515-b5cc-c055fca6da73,sp-traffic,ATVPDKIKX0DER,USD,A2UQYYL1FV7LGU,185402650650808,230449422777252,167430181164441,25420145967171,substitutes,TARGETING_EXPRESSION_PREDEFINED,Detail Page on-Amazon,2022-09-10T04:00:00-07:00,0,-7,0.00,2022,9,12


In [35]:
import awswrangler as wr

databases = wr.catalog.databases()

sp_traffic_db = 'stream-sp-traffic'
sp_traffic_table = 'sp-traffic'

if sp_traffic_db not in databases.values:
    wr.catalog.create_database(sp_traffic_db)
    print(wr.catalog.databases())
else:
    print("Database stream-sp-traffic already exists")

wr.s3.to_parquet(
    df=df,
    path="s3://map-migrated-wj-test/cost-analyzer-glue/",
    dataset=True,
    database=sp_traffic_db,
    table=sp_traffic_table,
    partition_cols=['year','month','day']
)

                                  Database            Description
0  athenacurcfn_map_migrated_report_cn_hwj                       
1                              aurora-glue                       
2                       cost_analyzer_glue                       
3                                  default  Default Hive database
4                         market_stream_db                       
5                         stream-sp-traffi                       
6                        stream-sp-traffic                       
7                                     test                       


{'paths': ['s3://map-migrated-wj-test/cost-analyzer-glue/year=2022/month=9/day=12/8154e508a97548d0a3013c717e07792c.snappy.parquet'],
 'partitions_values': {'s3://map-migrated-wj-test/cost-analyzer-glue/year=2022/month=9/day=12/': ['2022',
   '9',
   '12']}}

In [37]:
client.close()